In [10]:
import pandas as pd
import numpy as np

# Demo data from Cloud Drive

In [11]:
# read data
viewdeos = pd.read_excel('file_name.xlsx', sheet_name='practice')
#viewdeos.info()
#viewdeos.head(20)
#viewdeos.tail(5)

In [12]:
# clear duplicate row
viewdeos = viewdeos.drop([45, 46, 47])
# slicing 'column_1'
# replace "." into space
viewdeos['column_1'] = viewdeos['column_1'].str.replace('.', ' ', regex=False)
#viewdeos['column_1'] = viewdeos['column_1'].str.replace('-', '', regex=False)
viewdeos['column_1'] = viewdeos['column_1'].str.split().str.get(0)
#viewdeos.head(20)

In [13]:
# sum up 'column_3' according to column_1
column_3_viewdeos = viewdeos.groupby(['column_1'])['column_3'].agg('sum')
column_3_viewdeos = pd.concat([column_3_viewdeos], axis=1).reset_index()
# convert float to str in order to compare the difference later
column_3_viewdeos['column_3'] = column_3_viewdeos['column_3'].astype(int)
column_3_viewdeos['column_3'] = column_3_viewdeos['column_3'].astype(str)
# to change column name
column_3_viewdeos.rename(columns={'column_1':'Website', 'column_3':'Cloud Impression'}, inplace=True)
#column_3_viewdeos.info()
#column_3_viewdeos

# Compare demo data from Backyard

In [14]:
# read data
backyard = pd.read_excel("viewdeos_compare.xlsx")
#backyard.info()
#backyard.head(20)
#backyard.tail(5)

In [15]:
# clear total row(without website name)
backyard = backyard.drop([1083])
# slicing 'column_2'
backyard['column_2'] = backyard['column_2'].str.replace(".", " ", regex=False)
backyard['column_2'] = backyard['column_2'].str.split().str.get(0)

In [16]:
# sum up 'Impression' according to 'column_2'
impression_backyard = backyard.groupby(['column_2'])['column_4'].agg('sum')
impression_backyard = pd.concat([impression_backyard], axis=1).reset_index()
# convert float to str in order to compare the difference later
impression_backyard['column_4'] = impression_backyard['column_4'].astype(str)
# to change column name
impression_backyard.rename(columns={'column_2':'Website', 'column_4':'Backyard Impression'}, inplace=True)
#impression_backyard.info()
#impression_backyard

# Find out different website based on total Impression

In [17]:
diff = impression_viewdeos[~impression_viewdeos.apply(tuple,1).isin(impression_backyard.apply(tuple,1))]
#print("data in Viewdeos different from data in Backyard:")
#diff

In [18]:
# Cloud vs backyard
temp = impression_viewdeos
def add_column(row):
    mystr = row['Website'].replace(" ", "")
    mystr = mystr.lower()
    
    if [mystr == impression_backyard['Website'].str.lower()]:
        return impression_backyard.loc[impression_backyard['Website'].str.lower()==mystr, 'Backyard Impression'].values
    
temp['Backyard Impression'] = temp.apply(add_column, axis=1)

temp

# convert ndarray in clomun 'Cloud Impression' to int
for i in range(temp['Backyard Impression'].size):
    if temp.loc[i, 'Backyard Impression'].size != 0:
        temp.loc[i, 'Backyard Impression'] = temp.loc[i, 'Backyard Impression'][0]
    else:
        temp.loc[i, 'Backyard Impression']=0
        
temp['Cloud Impression'] = temp['Cloud Impression'].astype(int)        
temp['Backyard Impression'] = temp['Backyard Impression'].astype(int)
#temp.info()
#temp

# find out the difference between cloud & Backyard
def add_difference(row):
    return row['Cloud Impression']-row['Backyard Impression']
temp['Cloud-Backyard'] = temp.apply(add_difference, axis=1)

# tag the status od the website in Backyard
def add_status(row):
    if row['Cloud-Backyard'] == 0:
        return "正常"
    elif row['Backyard Impression'] == 0:
        return "無數據"
    else:
        return "異常"
    
temp['Backyard Status'] = temp.apply(add_status, axis=1)
#temp